In [61]:
import csv
import selenium.webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
from tqdm import tqdm
from selenium import webdriver

In [62]:
from datetime import date
today = date.today()
# dd/mm/YY
d1 = today.strftime("%d/%m/%Y")
print("d1 =", d1)
# Textual month, day and year
d2 = today.strftime("%B %d, %Y")
print("d2 =", d2)
# mm/dd/y
d3 = today.strftime("%m/%d/%y")
print("d3 =", d3)
# Month abbreviation, day and year
d4 = today.strftime("%b-%d-%Y")
print("d4 =", d4)

d1 = 28/11/2019
d2 = November 28, 2019
d3 = 11/28/19
d4 = Nov-28-2019


In [63]:
# User defined variables for data retreival
origin = "BOM"  # Origin airport code
destin = "DEL"  #Destination airport code
import sys # for getting sys date
trDate = sys.argv[1] # Date as 1st command line argument.

In [9]:
""" The following is the Base Url for fetching data from MakeMyTrip Website.
This URL appears in the search bar after origin, destination and date inputs on the landing page.
Thus, this URL can be changed based on User Inputs and required data can be fetched.
"""

' The following is the Base Url for fetching data from MakeMyTrip Website.\nThis URL appears in the search bar after origin, destination and date inputs on the landing page.\nThus, this URL can be changed based on User Inputs and required data can be fetched.\n'

In [64]:
baseDataUrl = "https://www.makemytrip.com/flight/search?tripType=O&itinerary=BOM-DEL-30/11/2019&paxType=A-1_C-0_I-0&cabinClass=E&sTime=1574881792216&forwardFlowRequired=true&intl=false"

In [30]:
driver = selenium.webdriver.Firefox()# firefox driver is being used.
print ("Requesting URL: " + baseDataUrl)

Requesting URL: https://www.makemytrip.com/flight/search?tripType=O&itinerary=BOM-DEL-30/11/2019&paxType=A-1_C-0_I-0&cabinClass=E&sTime=1574881792216&forwardFlowRequired=true&intl=false


In [31]:
driver.get(baseDataUrl)  # URL requested in browser.
print ("Webpage found ...")

Webpage found ...


In [65]:
driver=webdriver.Firefox()
driver.get(baseDataUrl)
html=driver.page_source
soupBody=BeautifulSoup(html,'html.parser')

UnexpectedAlertPresentException: Alert Text: None
Message: Dismissed user prompt dialog: The proxy moz-proxy://172.30.52.18:8080 is requesting a username and password. The site says: “WebMarshal Proxy Server”


In [66]:
driver.get(baseDataUrl)
html=driver.page_source
soupBody=BeautifulSoup(html,'html.parser')

In [68]:
element_xpath = '//*[@id="left-side--wrapper"]/div[2]' # First box with relevant flight data.

# Wait until the first box with relevant flight data appears on Screen
element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

# Scroll the page till bottom to get full data available in the DOM.
print ("Scrolling document upto bottom ...")
for j in range(1, 1000):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Find the document body and get its inner HTML for processing in BeautifulSoup parser.
body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

"""print("Closing Chrome ...") # No more usage needed.
driver.quit() # Browser Closed.

print("Getting data from DOM ...")
soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup"""

Scrolling document upto bottom ...


'print("Closing Chrome ...") # No more usage needed.\ndriver.quit() # Browser Closed.\n\nprint("Getting data from DOM ...")\nsoupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup'

In [75]:
# Extract the required tags 
spanFlightName = soupBody.findAll("div", {"class": "pull-left airways-info-sect"}) # Tags with Flight Name
pFlightCode = soupBody.findAll("p", {"class": "fli-code"})  # Tags with Flight Code
divDeptTime = soupBody.findAll("div", {"class": "dept-time"})  # Tags with Departure Time
pDeptCity = soupBody.findAll("p", {"class": "dept-city"})    # Tags with Departure City
pFlightDuration = soupBody.findAll("p", {"class": "fli-duration"}) # Tags with Flight Duration
pArrivalTime = soupBody.findAll("p", {"class": "reaching-time append_bottom3"})  # Tags with Arrival Time
pArrivalCity = soupBody.findAll("p", {"class": "arrival-city"})  # Tags with Arrival City
spanFlightCost = soupBody.findAll("span", {"class": "actual-price"}) # Tags with Flight Cost

In [78]:
spanFlightName

[]

In [77]:
'''# Data Headers
flightsData = ["flight_name", #"flight_code", "departure_time", "departure_city", "flight_duration", "arrival_time", "arrival_city", "flight_cost"]

# Extracting data from tags and appending to main database flightsData
for j in range(0, len(spanFlightName)):
    flightsData.append([spanFlightName[j].text, pFlightCode[j].text, divDeptTime[j].text, pDeptCity[j].text, pFlightDuration[j].text, pArrivalTime[j].text, pArrivalCity[j].text, spanFlightCost[j].text])'''


SyntaxError: invalid syntax (<ipython-input-77-6d95006639de>, line 2)

In [72]:
flightsData

[['flight_name',
  'flight_code',
  'departure_time',
  'departure_city',
  'flight_duration',
  'arrival_time',
  'arrival_city',
  'flight_cost']]

In [73]:

# Data Headers
flightsData = [["flight_name", "flight_code", "departure_time", "departure_city", "flight_duration", "arrival_time", "arrival_city", "flight_cost"]]

# Extracting data from tags and appending to main database flightsData
for j in range(0, len(spanFlightName)):
    flightsData.append([spanFlightName[j].text, pFlightCode[j].text, divDeptTime[j].text, pDeptCity[j].text, pFlightDuration[j].text, pArrivalTime[j].text, pArrivalCity[j].text, spanFlightCost[j].text])

In [59]:
flightsData

[['flight_name',
  'flight_code',
  'departure_time',
  'departure_city',
  'flight_duration',
  'arrival_time',
  'arrival_city',
  'flight_cost']]

In [74]:
# Output File for FlightsData. This file will have the data in comma separated form.
#outputFile =[["FlightsData_" + origin +"-"+ destin +"-"+ trDate.split("/")[0] + "-" + trDate.split("/")[1] + "-" + trDate.split("/")[2] + ".csv"]]

'''# Publishing Data to File
print("Writing flight data to file: "+ outputFile + " ...")
with open(outputFile, 'w', newline='') as spfile:
    csv_writer = csv.writer(spfile)
    csv_writer.writerows(flightsData)
print ("Data Extracted and Saved to File. ")'''

IndexError: list index out of range